<I>**NOTE: This notebook contains failed experiment on fine-tuning the GPT-2 model for generating stories based on a given prompt. However, I will keep it here for reminder of a learning experience of the fact that even though many ideas related to fine-tuning language models seem to work at conceptual level, the reality can be quite different. In this case, I hypothesize that GPT-2 Medium (345M parameters) is not able to find the signal of the mapping in the training data. While a larger model might be able to do this, my computational budget for this project does not allow for testing this possibility.**</I>

# Decaying prompt control signal in text generation with GPT-2

This notebook tries to demonstrate with experiments how the control signal of instruction prompt decays as the distance increases between the prompt and the generated text. In the experiments, GPT-2 is first fine-tuned to generate stories based on a prompt the story should be related to. Then, in the evaluation phase the fine-tuned model is given a prompt to generate a "positive story" with varying lengths of story beggining sentences with negative sentiments. A sentiment classifier is then used to evaluate the sentiments of these generated stories and the probability of the positive sentiment is stored for each model generations at the varying lengths of the start of stories. The target of the experiments is to see whether the prompt signal instructing a positive sentiment decreases as the distance increases and text with negative sentiment is added.

As the dataset for fine-tuning GPT-2 for story generation and for obtaining the story beginnings, we will be using the WritingPrompts dataset [1]. The dataset is originally collected from Reddit's [WritingPrompts forum](https://www.reddit.com/r/WritingPrompts/), where users can respond with stories to story prompts provided by other users. From this dataset, we will be using the stories written by the users to fine-tune GPT-2 to generate stories according an instruction prompt. More specifically, in the fine-tuning and experiments stage we will utilize the following prompt: "Write a story related to the following topic: <|topic|>.<|INS|><|story|>", where <|topic|> is the prompt from the WritingPrompts dataset, <|INS|> is a special token separating the instruction and the story, and <|story|> is a varying length start of a story from the WritingPrompts dataset.

### Experiment setup
Text Generation Model: [GPT-2 Medium](https://huggingface.co/openai-community/gpt2-medium) (355M parameters)<br>
Sentiment Classifier Model: [Twitter-roBERTa-base for Sentiment Analysis](https://huggingface.co/cardiffnlp/twitter-roberta-base-sentiment-latest) (125M parameters)<br>
Dataset: [WritingPrompts](https://github.com/facebookresearch/fairseq/blob/main/examples/stories/README.md)

## Experiments

We start by first installing the needed libraries. We will use [Hugging Face](https://huggingface.co/) for downloading the pre-trained models, GPT-2 and fine-tuned RoBERTa, and [PyTorch](https://pytorch.org/) for fine-tuning GPT-2 to generate text according to the instructed sentiment.

In [21]:
%pip install torch transformers peft numpy

Note: you may need to restart the kernel to use updated packages.


In [85]:
import torch
from transformers import GPT2TokenizerFast, GPT2LMHeadModel, GenerationConfig, RobertaTokenizerFast, RobertaForSequenceClassification
from transformers import get_linear_schedule_with_warmup
from peft import LoraConfig, get_peft_model
from torch.utils.data.dataset import Dataset
from torch.utils.data.dataloader import DataLoader
import numpy as np
import re

torch_device = "cuda" if torch.cuda.is_available() else "cpu"

### Data pre-processing
After importing the libraries, we will download the dataset and preprocess it for the fine-tuning and evaluation stages. To download the dataset, we follow the instructions given in the [README.md](https://github.com/facebookresearch/fairseq/blob/main/examples/stories/README.md) of the Hierarchical Neural Story Generation, which is the paper where the WritingPrompts dataset was presented.

In [5]:
!curl https://dl.fbaipublicfiles.com/fairseq/data/writingPrompts.tar.gz | tar xvzf -

  % Total    % Received % Xferd  Average Speed   Time    Time     Time  Current
                                 Dload  Upload   Total   Spent    Left  Speed
  0     0    0     0    0     0      0      0 --:--:-- --:--:-- --:--:--     0x writingPrompts/
x writingPrompts/test.wp_source
  3  363M    3 11.5M    0     0  6378k      0  0:00:58  0:00:01  0:00:57 6376k
x writingPrompts/README
x writingPrompts/valid.wp_source
  8  363M    8 30.3M    0     0  8050k      0  0:00:46  0:00:03  0:00:43 8049k
 95  363M   95  346M    0     0  9121k      0  0:00:40  0:00:38  0:00:02  9.9M00:44  0:00:05  0:00:39 9664k
100  363M  100  363M    0     0  9156k      0  0:00:40  0:00:40 --:--:-- 10.0M



In the experiments, we will use the target training dataset to fine-tune the GPT-2 model for generating stories, the validation dataset for checking model performance during the training, and the test dataset for giving the initial context in the control signal evaluation phase. As the datasets are unnecessary large for our purposes, we will only take a subset of the stories from each. More specifically, we will only fine-tune the model with the stories having prompts of under 100 characters, and the first 20,000 stories of these.

In [86]:
def load_file(filename: str, encoding="utf-8") -> list[str]:
    with open(filename, "r", encoding=encoding) as f:
        return f.readlines()

In [87]:
def filter_data(prompts: list[str], stories: list[str], max_length: int) -> tuple[list[str], list[str]]:
    short_prompts = [prompt for prompt in prompts if len(prompt) < max_length]
    story_idx = [idx for idx, prompt in enumerate(prompts) if len(prompt) < max_length]
    target_stories = np.array(stories)[story_idx].tolist()
    return short_prompts, target_stories

In [88]:
train_prompts = load_file("writingPrompts/train.wp_source")[:100_000] # take only subset due to memory issues
train_stories = load_file("writingPrompts/train.wp_target")[:100_000] # take only subset due to memory issues
valid_prompts = load_file("writingPrompts/valid.wp_source")
valid_stories = load_file("writingPrompts/valid.wp_target")
test_prompts = load_file("writingPrompts/test.wp_source")
test_stories = load_file("writingPrompts/test.wp_target")

train_prompts, train_data = filter_data(train_prompts, train_stories, 100)
train_prompts, train_data = train_prompts[:20_000], train_data[:20_000]
valid_prompts, valid_data = filter_data(valid_prompts, valid_stories, 100)
valid_prompts, valid_data = valid_prompts[:1000], valid_data[:1000]
test_prompts, test_data = filter_data(test_prompts, test_stories, 100)
test_prompts, test_data = test_prompts[:1000], test_data[:1000]

print(f"Filtered train data: {len(train_data)}")
print(f"Filtered validation data: {len(valid_data)}")
print(f"Filtered test data: {len(test_data)}")

Filtered train data: 20000
Filtered validation data: 1000
Filtered test data: 1000


In  what follows, we use the [Hugging Face Tokenizer](https://huggingface.co/docs/transformers/en/main_classes/tokenizer) for tokenizing the train, validation, and test datasets. As we use GPT-2 (medium) as our language model, we will use the GPT2TokenizerFast. Underneath the class abstractions, OpenAI models utilize the [BPE (Byte pair encoding) algorithm](https://en.wikipedia.org/wiki/Byte_pair_encoding) in tokenizing the input texts for its models. For GPT-2, this encoding has a vocabulary size of 50,257 tokens. Before tokenizing, we have to also replace the special token of `<newline>` in the datasets to the standard `\n` token indicating a new line in the text. In addition, we will truncate all the stories including the instruction prompt to 512 tokens as the RoBERTa-based sentiment classifier can only classify sequences up to 512 tokens (different encoding algorithm but ballpark is the same), thus making it unnecessary to fine-tune the GPT-2 to generate longer stories. This will also reduce the memory requirements. Shorter sequences than 512 are padded with the `<|endoftext|>` token.

We will utilize a custom PyTorch DataSet class for constructing the training and validation datasets, and the PyTorch DataLoader for enabling the batching (batch_size=6) and shuffling (shuffle=True) required by the mini-batch stochastic gradient descent. As the fine-tuning dataset is quite large, we will tokenize the stories only after they are selected to the current mini-batch. Therefore, the tokenization is done in a custom collate function of the DataLoader to enable tokenizing the whole batch in one go.

In [95]:
instruction_token = "<|INS|>"
gpt2_tokenizer = GPT2TokenizerFast.from_pretrained("gpt2", pad_token="<|endoftext|>")
gpt2_tokenizer.add_special_tokens({"additional_special_tokens": [instruction_token]})

1

In [96]:
class StoryGenerationDataset(Dataset):
    def __init__(self, prompts: list[str], stories: list[str]):
        self.prompts = [re.sub("\[[^\]]*\]", "", prompt).replace("<newline>", "\n").strip() for prompt in prompts]
        self.stories = [story.replace("<newline>", "\n").strip() for story in stories]

    def __len__(self):
        return len(self.prompts)

    def __getitem__(self, idx: int):
        prompt = self.prompts[idx]
        story = self.stories[idx]
        return f"Write a story related to the following topic: {prompt}{instruction_token}{story}"

In [97]:
def collate_batch(batch: list[str], tokenizer: GPT2TokenizerFast, max_length: int = 512):
    encodings = tokenizer(text=batch, max_length=max_length, truncation=True, padding="max_length", return_tensors="pt").to(torch_device)
    return encodings["input_ids"], encodings["attention_mask"]

In [80]:
gpt2_tokenizer = GPT2TokenizerFast.from_pretrained("gpt2", pad_token="<|endoftext|>")
gpt2_tokenizer.add_special_tokens({"additional_special_tokens": ["<|INS|>"]})

batch_size = 6

train_dataset = StoryGenerationDataset(prompts=train_prompts, stories=train_data)
train_dataloader = DataLoader(
    dataset=train_dataset,
    batch_size=batch_size,
    shuffle=True,
    collate_fn=lambda x: collate_batch(x, gpt2_tokenizer, max_length=512)
)

valid_dataset = StoryGenerationDataset(prompts=valid_prompts, stories=valid_data)
valid_dataloader = DataLoader(
    dataset=valid_dataset,
    batch_size=batch_size,
    shuffle=True,
    collate_fn=lambda x: collate_batch(x, gpt2_tokenizer, max_length=512)
)

### Fine-tuning

Now that we have our fine-tuning dataset and corresponding data loader ready, we can formulate the training loop for the fine-tuning process. For this we need to instantiate the GPT-2 Medium model object. We will use the Hugging Face model instantiation for this, openai-community/gpt2-medium. For the hyperparameters, we will use as the a starting learning rate `0.001` with a linearly decreasing schedule with warmup, warmup steps `100`, and epsilon `1e-8`. In total we will fine-tune for three epochs. At every 100 step we will generate a sample story text and also print out the validation loss, which is expected to decrease as the model adapts to the story domain. As the optimizer, we will use the AdamW-algorithm optimizer implemented by Hugging Face. We will also utilize gradient accumulation to simulate the situation where the batch size would be larger by accumulating the gradients over 80 steps before updating the weights with the optimizer.

In [98]:
gpt2_model = GPT2LMHeadModel.from_pretrained("openai-community/gpt2-medium").to(torch_device)
gpt2_model.resize_token_embeddings(len(gpt2_tokenizer))

lr = 0.001 # use smaller learning rate for fine-tuning
eps = 1e-8
num_epochs = 3
num_warmup = 100
accumulation_steps = 80 # emulate batch size of 480 by taking the backward pass only every 80 mini-batch

optimizer = torch.optim.AdamW(gpt2_model.parameters(), lr=lr, eps=eps)
total_steps = len(train_dataloader) * num_epochs

scheduler = get_linear_schedule_with_warmup(optimizer, num_warmup_steps=num_warmup, num_training_steps=total_steps)

generation_config = GenerationConfig(
    max_new_tokens=40,
    do_sample=True,
    top_k=40,
    pad_token_id=gpt2_tokenizer.pad_token_id,
)

sample_input_prompt = "Write a story related to the following topic: 'an overly positive story with happy ending'.<|INS|>"
sample_input = gpt2_tokenizer(sample_input_prompt, return_tensors="pt").to(torch_device)

In [99]:
def generate_sample_text_during_training(model, tokenizer, input: dict, generation_config: GenerationConfig) -> None:
    model.eval()

    with torch.no_grad():
        sample_output = model.generate(
            inputs=input["input_ids"],
            attention_mask=input["attention_mask"],
            generation_config=generation_config
        )
        sample_text = tokenizer.decode(sample_output[0], skip_special_tokens=True)
        print(f"Sample output: {sample_text}")

    model.train()

After the initial try I realized that the fine-tuning process with only a subset of the full data will take a substantial time even with an A100 GPU. Luckily, there exists numerous methods to speed-up the fine-tuning process; often referred to as parameter-efficient fine-tuning (PEFT) methods. From these methods, we will utilize the LoRA (Low-Rank Adaptation) method to decrease the number of trainable parameters in the model. To apply LoRA in to the training process, the [PEFT-library](https://huggingface.co/docs/peft/en/index) by Hugging Face will be utilized.

In [100]:
config = LoraConfig(
    r=16, # rank
    lora_alpha=16, # scaling factor (more in the original paper)
    lora_dropout=0.1, # regularization parameter, to be honest this is chosen arbitrarily, usually 0.05 is a reasonable value
    bias="none", # bias is not used in the original paper
    task_type="CAUSAL_LM", # Note: add modules_to_save if you want to include in the fine-tuning some other modules,
    fan_in_fan_out=True,
    modules_to_save=["wte"] # train special tokens embeddings
)
lora_model = get_peft_model(gpt2_model, config)
lora_model.print_trainable_parameters()

trainable params: 53,037,056 || all params: 407,861,248 || trainable%: 13.0037


In [ ]:
%%time

start_time = time.time()
train_losses = []

for epoch in range(num_epochs):
    print(f"Epoch {epoch + 1}/{num_epochs}")

    total_train_loss = 0
    lora_model.train()

    for step, batch in enumerate(train_dataloader):
        step_start_time = time.time()
        input_ids, attention_mask = batch
        outputs = lora_model(input_ids, attention_mask=attention_mask, labels=input_ids)
        loss = outputs.loss / accumulation_steps # apply the effect of the average over the mini batches in the gradient accumulation
        total_train_loss += loss.item()
        train_losses.append(loss.item())
        loss.backward() # use gradient accumulation

        if step % 100 == 0:
            step_end_time = time.time()
            print(f"Step {step} of {total_steps} - Loss: {loss.item()} - Time: {step_end_time - step_start_time}")

        if step % accumulation_steps == 0:
          optimizer.step()
          scheduler.step()
          optimizer.zero_grad()

        if step % 500 == 0:
          lora_model.eval()
          total_eval_loss = 0
          for batch in valid_dataloader:
            input_ids, attention_mask = batch
            with torch.no_grad():
                outputs = lora_model(input_ids, attention_mask=attention_mask, labels=input_ids)
            eval_loss = outputs.loss
            total_eval_loss += eval_loss.item()
          
          print(f"Average validation loss: {total_eval_loss / len(valid_dataloader)}")
          lora_model.train()

          # Generate a test sample
          generate_sample_text_during_training(lora_model, gpt2_tokenizer, sample_input, generation_config)

    print(f"Epoch 1: Average training loss: {total_train_loss / len(train_dataloader)}")
    epoch_time = time.time()
    print(f"Time from start: {epoch_time - start_time} seconds")

end_time = time.time()
print(f"Training completed successfully! Time: {end_time - start_time}")

### Saving the fine-tuned model
Next we save the fine-tuned model to disk and use it later when experimenting with the controlled text generation.

In [ ]:
# Merge the model weights to avoid inference latency
lora_model_merged = lora_model.merge_and_unload()
lora_model.save_pretrained("./artifacts/gpt2-medium-finetuned-story-generation-ctg", push_to_hub=False)
gpt2_tokenizer.save_pretrained("./artifacts/gpt2-medium-finetuned-story-generation-ctg", push_to_hub=False)

### Controlling the text generation

Now that the model has been fine-tuned for story generation based on an instruction prompt, we can move on to the controlled text generation phase of our experiments.

In [84]:
fine_tuned_gpt2 = GPT2LMHeadModel.from_pretrained("./artifacts/model", use_safetensors=True)

sample_input_prompt = "Write a story related to the following topic: 'an overly positive story with happy ending'.<|INS|>"
sample_input = gpt2_tokenizer(sample_input_prompt, return_tensors="pt").to(torch_device)

with torch.no_grad():
    sample_output = fine_tuned_gpt2.generate(
        inputs=sample_input["input_ids"],
        attention_mask=sample_input["attention_mask"],
        generation_config=generation_config
    )
    sample_text = gpt2_tokenizer.decode(sample_output[0], skip_special_tokens=False)
    print(f"Sample output: {sample_text}")

Sample output: Write a story related to the following topic: 'an overly positive story with happy ending'.<|INS|><|INS|><|INS|><|INS|><|INS|><|INS|><|INS|><|INS|><|INS|><|INS|><|INS|><|INS|><|INS|><|INS|><|INS|><|INS|><|INS|><|INS|><|INS|><|INS|><|INS|><|INS|><|INS|><|INS|><|INS|><|INS|><|INS|><|INS|><|INS|><|INS|><|INS|><|INS|><|INS|><|INS|><|INS|><|INS|><|INS|><|INS|><|INS|><|INS|><|INS|>


Based on the generated text above and other experiment results, the GPT-2 model has not been able to learn the task of generating a story based on the prompt. My hypothesis for this is that the mapping of the generic prompt to some generic story is too complicated for GPT-2 to capture as the model has reasonably few weights. This is also seen in the fact that the training loss did not decrease at all during the fine-tuning process. Due to not being able to fine-tune GPT-2 for story generation, I will not continue with this branch of experiments related to investigating the controlled text generation of these language models.

## References
[1] A. Fan, M. Lewis, and Y. Dauphin. Hierarchical neural story generation. In ACL 2018 - 56th Annual Meeting of the Association for Computational Linguistics, Proceedings of the Conference (Long Papers), volume 1, page 889 – 898, 2018.